In [4]:
import numpy as np 
import random

## Greedy  KP

In [5]:
v = np.array([2,6,8,7,3,4,6,5,10,9,8,11,12,15,6,8,13,14,15,16,13,14,15,26,13,9,25,26])
w = np.array([7,3,3,5,4,7,5,4,15,10,17,3,6,11,6,14,4,8,9,10,14,17,9,24,11,17,12,14])
W = 30

In [6]:
# Calculate value of score 
def getscore(v,w,n):
    arr = np.array([])
    for i in range(n):
        arr = np.append(arr,round((v[i]/w[i]),3)) 
    score = np.argsort(-arr)  
    return score

def GKP(v,w,n,W):
    
    Totalval = 0    
    #Sort items 
    value =  getscore(v,w,n)
    kpelements = np.zeros(n).astype(int)   
    
    for i in range(n):
        kplarge = value[i]
        if((Totalval+w[kplarge]) <= W ):
            kpelements[kplarge] = 1
            Totalval += w[kplarge]
              
    #return pairs giving maximum profit
    return kpelements


In [7]:
selecteditems =GKP(v,w,28,30)
print("w :" ,w[selecteditems == 1])
print("v :", v[selecteditems == 1])
print("Profit :" , np.sum(v[selecteditems==1]))
print("Weight :" , np.sum(w[selecteditems==1]))

w : [ 3  3  5  3  4 12]
v : [ 6  8  7 11 13 25]
Profit : 70
Weight : 30


## PTAS  KP

In [ ]:
 def getsubsets(totsets, subset, current, k):
    if(len(subset)== current):
        return totsets
    
    newsubset = []
    for i in range(len(totsets)):        
        if(len(totsets[i]) < k):
            newsubset = totsets[i].copy()
            newsubset.append(subset[current])
            totsets.append(newsubset)
           
    getsubsets(totsets,subset,current+1 , k)
           
def capacityexceeded(currentw,W):           
    totw = np.sum(currentw)
    if(totw > W):
        return True
    else:
        return False

def getsubsetsize(arr,size, k):
    totsets = [[]]
    getsubsets(totsets,arr,0,k)
    finalset = []
    for sets in totsets:
        if (len(sets)== size) :
            finalset.append(sets)
    return finalset
           
def getsubsetsrange(arr, minsize, k):
    totsets = [[]]
    getsubsets(totsets,arr,0,k)
    finalset = []
    for sets in totsets:
        if (len(sets) >= minsize) :
            finalset.append(sets)
    return finalset

def PTAS(v,w,n,W,minsize,k):
    currW = 0
    fW = 0
    
    totsets = [[]]
    setindx = []
    
    for i in range(0,28):
        setindx.append(i)
    
    
    totsets = getsubsetsrange(setindx, minsize,k)
    totsets = np.array(totsets)

    maxprofit = 0
    bestIndx =[]
    bestLindx =[]
    
    # Pack the Knapsack
    for sets in totsets:      
        currentv = np.take(v,sets)
        currentw = np.take(w,sets)  
        if(capacityexceeded(currentw,W) == False):
            newv = np.take(v,list(set(setindx) - set(sets)))
            neww = np.take(w,list(set(setindx) - set(sets)))        
            totprofitPTAS = np.sum(currentv)
            totweight = np.sum(currentw)
            currentcapacity = W - totweight          
            kpelements = GKP(newv, neww, len(newv), currentcapacity)
            totprofit = totprofitPTAS + np.sum(newv[kpelements == 1])
            if(totprofit > maxprofit):
                maxprofit = totprofit
                bestIndx = sets
                bestLindx = kpelements
                nextv = newv
                nextw = neww
    
    #Use greedy to fill remaining capacity
    bestLvalues = np.where(bestLindx == 1)
    
    highestvalV = np.concatenate([np.take(v,bestIndx),np.take(nextv,bestLvalues)[0]])
    highestvalW = np.concatenate([np.take(w,bestIndx),np.take(nexttw,bestLvalues)[0]])
     
    # Return Highest valued combination set
    return highestvalV, highestvalW


In [ ]:
minsize = 3
k =10
bestv, bestw = PTAS(v,w,28,30,minsize,k)
print("w :" ,bestv)
print("v :", bestw)
print ("Profit :" , np.sum(bestv))

w :  [3   3  8  4  12]   


v :  [8   11  14  13  25]

Profit :  71 

## Quadratic KP

In [ ]:
def getrandom(k,N, snumbers):
    unumbers = np.delete(range(N), snumbers)
    
    if(len(unumbers) < k):
        norandom = len(unumbers)
    else:
        norandom = k
    randomindx = np.random.choice(unumbers , norandom)
    return randomindx

def sortitems(arr,p,w):
    efficeincies = []
    efficeincies = np.array(efficeincies)
    for pair in arr:
        P_ij = p[pair[0]][pair[1]]
        w_i = w[pair[0]]
        w_j = w[pair[0]]
        eff = P_ij/(w_i + w_j)
        efficeincies = np.append(efficeincies , eff)
    effindx = np.argsort(-efficeincies)
    return effindx

def sortitems2(arr,p,w):
    efficeincies = []
    efficeincies = np.array(efficeincies)
    for pair in arr:
        P_ij = p[pair[0]][pair[1]]
        w_i = w[pair[0]]
        w_j = w[pair[0]]
        v_i = v[pair[0]]
        v_j = v[pair[0]]
        eff = (P_ij + v_i +v_j) /(w_i + w_j)
        efficeincies = np.append(efficeincies , eff)
    effindx = np.argsort(-efficeincies)
    return effindx
    

def checkelements(newpair , chosenpair):
    firstindx = newpair[0]
    secondindx = newpair[1]
    
    if(firstindx in chosenpair or secondindx in chosenpair):
        return True
    else:
        return False
    
def QKP(k, N,randomlist , v, w, W, p):
    np.random.seed(randomlist)
    elementsindx = []
    pairsize = 2
    totsum = 0
    totprofit = 0
    setindx = []
    for i in range(0,15):
        setindx.append(i)
    
    maxiter = 1000
    noiter = 0
    
    while (noiter < maxiter):
        noiter+=1
        randomnoindx = getrandom(k,15,elementsindx)
        pairarr = getsubsetsize(randomnoindx ,pairsize, k)
        efficiencyindx = sortitems(pairarr,p,w)     
        effscore = np.argsort(efficiencyindx)
        
        curr = pairarr[efficiencyindx[0]]
        firstindx = curr[0]
        secondindx = curr[1]
        
        w_i = w[firstindx]
        w_j = w[secondindx]
        
        usedcapacity = totsum +w_i +w_j
        
        if(usedcapacity <= W):
            if(checkelements(curr,elementsindx)== False):
                elementsindx.append(firstindx)
                elementsindx.append(secondindx)
                newv = np.copy(v)
                neww = np.copy(w)
                np.put(newv , curr,0)
                totsum += w_i+w_j
                totprofit += (v[firstindx] + v[secondindx])
                W = W- totsum
                
        else:
            continue
            
        chosenv = np.take(v , elementsindx)
        chosenw = np.take(w , elementsindx)
        
        elementschosenindx = GKP (newv,neww, len(newv) , W)
        lindx = np.where(elementschosenindx==1)[0]
        totindx = np.concatenate(( lindx ,elementschosenindx ))
        totpairs = getsubsetsize(totindx, pairsize, N)
        profitperitem = np.sum(np.concatenate([np.take(v,elementsindx), newv[elementschosenindx ==1]]))
        finalprofit = profitperitem
        
        for i in range(len(totpairs)):
            newpair = totpairs[i]
            finalprofit += p[newpair[0]][newpair[1]]
            
        
        print("chosen v:" , chosenv)
        print("chosen w:", chosenw)   
        print("Total Profit :",profitperitem)
        
def QKP2(k, N,randomlist , v, w, W, p):
    np.random.seed(randomlist)
    elementsindx = []
    pairsize = 2
    totsum = 0
    totprofit = 0
    setindx = []
    for i in range(0,15):
        setindx.append(i)
    
    maxiter = 1000
    noiter = 0
    
    while (noiter < maxiter):
        noiter+=1
        randomnoindx = getrandom(k,15,elementsindx)
        pairarr = getsubsetsize(randomnoindx ,pairsize, k)
        # Second case using new efficiency function
        efficiencyindx = sortitems2(pairarr,p,w)     
        effscore = np.argsort(efficiencyindx)
        
        curr = pairarr[efficiencyindx[0]]
        firstindx = curr[0]
        secondindx = curr[1]
        
        w_i = w[firstindx]
        w_j = w[secondindx]
        
        usedcapacity = totsum +w_i +w_j
        
        if(usedcapacity <= W):
            if(checkelements(curr,elementsindx)== False):
                elementsindx.append(firstindx)
                elementsindx.append(secondindx)
                newv = np.copy(v)
                neww = np.copy(w)
                np.put(newv , curr,0)
                totsum += w_i+w_j
                totprofit += (v[firstindx] + v[secondindx])
                W = W- totsum
                
        else:
            continue
            
        chosenv = np.take(v , elementsindx)
        chosenw = np.take(w , elementsindx)
        
        elementschosenindx = GKP (newv,neww, len(newv) , W)
        lindx = np.where(elementschosenindx==1)[0]
        totindx = np.concatenate(( lindx ,elementschosenindx ))
        totpairs = getsubsetsize(totindx, pairsize, N)
        profitperitem = np.sum(np.concatenate([np.take(v,elementsindx), newv[elementschosenindx ==1]]))
        finalprofit = profitperitem
        
        for i in range(len(totpairs)):
            newpair = totpairs[i]
            finalprofit += p[newpair[0]][newpair[1]]
            
         
        print("Profit :",profitperitem)
        print("Weight :" ,np.sum(np.concatenate(chosenw,neww[elementschosenindx ==1])))
        

In [ ]:
np.array([7, 6, 13,16, 5, 10, 9, 23, 18, 12, 9, 22, 17, 32, 8])
w = np.array([13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
W = 50
k=7
N=15
randomlist = 7
p = np.array([
7   ,12 ,7  ,6  ,13 ,8  ,11 ,7  ,15 ,23 ,14 ,15 ,17 ,9  ,15,
12  ,6  ,15 ,13 ,10 ,15 ,9  ,10 ,8  ,17 ,11 ,13 ,12 ,16 ,15,
7   ,15 ,13 ,11 ,16 ,6  ,8  ,14 ,13 ,4  ,14 ,8  ,15 ,9  ,16,
6   ,13 ,11 ,16 ,10 ,13 ,14 ,14 ,17 ,15 ,14 ,6  ,24 ,13 , 4,
13  ,10 ,16 ,10 ,5  ,9  ,7  ,25 ,12 ,6  ,6  ,16 ,10 ,15 ,14,
8   ,15 ,6  ,13 ,9  ,10 ,2  ,13 ,12 ,16 ,9  ,11 ,23 ,10 ,21,
11  ,9  ,8  ,14 ,7  ,2  ,9  ,8  ,18 ,4  ,13 ,14 ,14 ,17 ,15,
7   ,10 ,14 ,14 ,25 ,13 ,8  ,23 ,9  ,16 ,12 ,3  ,14 ,14 ,27,
15  ,8  ,13 ,17 ,12 ,12 ,18 ,9  ,18 ,15 ,16 ,13 ,14 ,7  ,17,
23  ,17 ,4  ,15 ,6  ,16 ,4  ,16 ,15 ,12 ,28 ,5  ,19 ,6  ,18,
14  ,11 ,14 ,14 ,6  ,9  ,13 ,12 ,16 ,28 ,9  ,13 ,4  ,13 ,16,
15  ,13 ,8  ,6  ,16 ,11 ,14 ,3  ,13 ,5  ,13 ,22 ,11 ,19 ,13,
17  ,12 ,15 ,24 ,10 ,23 ,14 ,14 ,14 ,19 ,4  ,11 ,17 ,15 ,12,
9   ,16 ,9  ,13 ,15 ,10 ,17 ,14 ,7  ,6  ,13 ,19 ,15 ,32 ,16,
15  ,15 ,16 ,4  ,14 ,21 ,15 ,27 ,17 ,18 ,16 ,13 ,12 ,16 , 8
    ])
p = p.reshape(15,15)
QKP(k,N,randomlist,v,w,W,p)
print("Case 2:")
QKP2(k,N,randomlist,v,w,W,p)

chosen v: [18  9  22  32]

cosen w: [13 9 12 12]

Total Profit: 162

Case 2:

Profit : 162 

Weight : 46